# ultimate tennis 

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage containing the table
url = "https://www.ultimatetennisstatistics.com/tournamentEvent?tournamentEventId=4626&tab=results"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table by its ID ("resultsTable")
    table = soup.find("table", {"id": "resultsTable"})

    if table:
        # Create an empty DataFrame to store the data
        data = []
        
        # Initialize a variable to keep track of the row index
        row_index = 0
        
        # Extract and append the data from the table to the DataFrame
        for row in table.find_all("tr"):
            player_cell = row.find("td", {"class": "player"})
            score_cells = row.find_all("td", {"class": "score"})

            if player_cell:
                player_name = player_cell.text.strip()
                # Extract nationality from the title attribute of the img tag
                nationality = player_cell.find("img")["title"]

                # Initialize variables for tiebreak sets and games in each set
                tiebreak_set1 = tiebreak_set2 = tiebreak_set3 = 0
                games_set1 = games_set2 = games_set3 = None

                if score_cells:
                    scores = [cell.text.strip() for cell in score_cells]

                    # Extract tiebreak information and update set scores
                    if len(scores) > 0 and "(" in scores[0]:
                        tiebreak_set1 = 1
                        games_set1 = int(scores[0].split("(")[1].split(")")[0])
                        scores[0] = scores[0].split("(")[0].strip()
                    if len(scores) > 1 and "(" in scores[1]:
                        tiebreak_set2 = 1
                        games_set2 = int(scores[1].split("(")[1].split(")")[0])
                        scores[1] = scores[1].split("(")[0].strip()
                    if len(scores) > 2 and "(" in scores[2]:
                        tiebreak_set3 = 1
                        games_set3 = int(scores[2].split("(")[1].split(")")[0])
                        scores[2] = scores[2].split("(")[0].strip()

                # Determine the 'stage' based on row index
                row_index += 1
                if row_index in [3, 4, 11, 12, 19, 20, 27, 28, 35, 36, 43, 44, 51, 52, 59, 60]:
                    stage = "quarterfinal"
                elif row_index in [5, 6, 13, 14, 21, 22, 29, 30]:
                    stage = "semifinal"
                elif row_index in [7, 8, 15, 16]:
                    stage = "final"
                else:
                    stage = "roundrobin"

                data.append([player_name, nationality, scores[0] if len(scores) > 0 else None, scores[1] if len(scores) > 1 else None, scores[2] if len(scores) > 2 else None, tiebreak_set1, tiebreak_set2, tiebreak_set3, games_set1, games_set2, games_set3, stage])

        # Create a Pandas DataFrame from the data
        df = pd.DataFrame(data, columns=["Player", "Nationality", "Set1", "Set2", "Set3", "Tiebreak_Set1", "Tiebreak_Set2", "Tiebreak_Set3", "Games_Set1", "Games_Set2", "Games_Set3", "Stage"])

        # Display the DataFrame
        print(df)

else:
    print("Failed to fetch the webpage. Status code:", response.status_code)


                   Player Nationality Set1 Set2  Set3  Tiebreak_Set1  \
0                 B. Gojo         CRO    6    3     6              0   
1                 B. Gojo         CRO    6    3     6              0   
2                 S. Baez         ARG    1    6     3              0   
3             J. Thompson         AUS    4    7     6              0   
4           T. Griekspoor         NED    6    5     3              0   
..                    ...         ...  ...  ...   ...            ...   
167         T. Griekspoor         NED    7    7  None              0   
168               T. Paul         USA    5    6  None              0   
169  B. Van De Zandschulp         NED    6    7     4              0   
170  B. Van De Zandschulp         NED    6    7  None              0   
171           T. H. Fritz         USA    4    6  None              0   

     Tiebreak_Set2  Tiebreak_Set3  Games_Set1  Games_Set2  Games_Set3  \
0                0              0         NaN         NaN     

In [8]:
df

,Player,Nationality,Set1,Set2,Set3,Tiebreak_Set1,Tiebreak_Set2,Tiebreak_Set3,Games_Set1,Games_Set2,Games_Set3,Stage
0,B. Gojo,CRO,6,3,6,0,0,0,NaN,NaN,NaN,roundrobin
1,B. Gojo,CRO,6,3,6,0,0,0,NaN,NaN,NaN,roundrobin
2,S. Baez,ARG,1,6,3,0,0,0,NaN,NaN,NaN,quarterfinal
3,J. Thompson,AUS,4,7,6,0,0,0,NaN,NaN,NaN,quarterfinal
4,T. Griekspoor,NED,6,5,3,0,0,0,NaN,NaN,NaN,semifinal
...,...,...,...,...,...,...,...,...,...,...,...,...
167,T. Griekspoor,NED,7,7,None,0,1,0,NaN,7.0,NaN,roundrobin
168,T. Paul,USA,5,6,None,0,1,0,NaN,3.0,NaN,roundrobin
169,B. Van De Zandschulp,NED,6,7,4,0,1,0,NaN,7.0,NaN,roundrobin
170,B. Van De Zandschulp,NED,6,7,None,0,1,0,NaN,7.0,NaN,roundrobin


In [9]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage containing the table
url = "https://www.ultimatetennisstatistics.com/tournamentEvent?tournamentEventId=4626&tab=results"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table by its ID ("resultsTable")
    table = soup.find("table", {"id": "resultsTable"})

    # Initialize a list to store the href links
    href_links = []

    if table:
        # Find all rows in the table
        rows = table.find_all("tr")

        # Loop through each row in the table
        for row in rows:
            # Find the specific "player" td element in the row
            player_cell = row.find("td", {"class": "player"})

            # Check if the player_cell is not None
            if player_cell:
                # Find the player's href link in the first column
                player_link = player_cell.find("a")

                # Check if the player_link is not None and get the href attribute
                if player_link:
                    href = player_link.get("href")
                    href_links.append(href)

    # Print the retrieved href links
    for link in href_links:
        print(link)

else:
    print("Failed to fetch the webpage. Status code:", response.status_code)


/playerProfile?playerId=27140
/playerProfile?playerId=27140
/playerProfile?playerId=46888
/playerProfile?playerId=11415
/playerProfile?playerId=34651
/playerProfile?playerId=6418
/playerProfile?playerId=6409
/playerProfile?playerId=33214
/playerProfile?playerId=6409
/playerProfile?playerId=6418
/playerProfile?playerId=6418
/playerProfile?playerId=46887
/playerProfile?playerId=45105
/playerProfile?playerId=22139
/playerProfile?playerId=45105
/playerProfile?playerId=5220
/playerProfile?playerId=44824
/playerProfile?playerId=45105
/playerProfile?playerId=11174
/playerProfile?playerId=11174
/playerProfile?playerId=46888
/playerProfile?playerId=6418
/playerProfile?playerId=5131
/playerProfile?playerId=32067
/playerProfile?playerId=33214
/playerProfile?playerId=44481
/playerProfile?playerId=44481
/playerProfile?playerId=6029
/playerProfile?playerId=5220
/playerProfile?playerId=5793
/playerProfile?playerId=44824
/playerProfile?playerId=52132
/playerProfile?playerId=11174
/playerProfile?player

In [10]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage containing the table
base_url = "https://www.ultimatetennisstatistics.com"
url = "https://www.ultimatetennisstatistics.com/tournamentEvent?tournamentEventId=4626&tab=results"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table by its ID ("resultsTable")
    table = soup.find("table", {"id": "resultsTable"})

    # Initialize a list to store the href links
    href_links = []

    if table:
        # Find all rows in the table
        rows = table.find_all("tr")

        # Loop through each row in the table
        for row in rows:
            # Find the specific "player" td element in the row
            player_cell = row.find("td", {"class": "player"})

            # Check if the player_cell is not None
            if player_cell:
                # Find the player's href link in the first column
                player_link = player_cell.find("a")

                # Check if the player_link is not None and get the href attribute
                if player_link:
                    href = player_link.get("href")
                    href_links.append(href)

    # Access each URL and retrieve information from the three tables
    for href in href_links:
        player_url = base_url + href
        player_response = requests.get(player_url)

        if player_response.status_code == 200:
            player_soup = BeautifulSoup(player_response.text, "html.parser")
            
            # Find the three tables with class "table table-condensed text-nowrap"
            tables = player_soup.find_all("table", {"class": "table table-condensed text-nowrap"})
            
            # Loop through the three tables
            for i, table in enumerate(tables):
                # Print or process information from each table
                print(f"Table {i + 1}:")
                for row in table.find_all("tr"):
                    # Process each row in the table as needed
                    # Example: Print the text content of each row
                    print(row.text.strip())
                print()  # Separate tables with a blank line
            
        else:
            print("Failed to fetch player's webpage. Status code:", player_response.status_code)

else:
    print("Failed to fetch the webpage. Status code:", response.status_code)


# Official Davis Cup website

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import re

# Initialize Selenium
chrome_service = ChromeService("C:/Users/ALESSANDRO/Downloads/chromedriver.exe")
chrome_service.start()
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# URL of the webpage
url = "https://www.daviscup.com/en/draws-results/tie.aspx?id=M-DC-2018-WG-M-FRA-NED-01"

# Navigate to the webpage
driver.get(url)

try:
    # Find the div element with class "main"
    main_element = driver.find_element(By.CLASS_NAME, "main")

    # Find the div element with class "rubber-header"
    rubber_header_element = main_element.find_element(By.CLASS_NAME, "rubber-header")

    # Extract "match" and "match status" from the span elements
    spans = rubber_header_element.find_elements(By.TAG_NAME, "span")
#     span_element = rubber_header_element.find_element(By.CSS_SELECTOR, "span.label.ng-binding")


    match_status = ""
    match = ""

    
    if len(spans) >= 2:
        match_status = spans[1].text.strip()
#         match = span_element.text.strip()

    # Find the div element with class "component-title ng-binding" within the main element
    component_title_element = main_element.find_element(By.CLASS_NAME, "component-title")

    # Include the component_title_text in the stage variable
    stage = component_title_element.text.strip()

    # Now, let's find the div element with class "tie" within the main element
    tie_element = main_element.find_element(By.CLASS_NAME, "tie")

    # Find all sub div elements within the "tie" element
    sub_div_elements = tie_element.find_elements(By.TAG_NAME, "div")

    # Initialize variables to store data
    column_data = {}

    for sub_div_element in sub_div_elements:
        sub_div_text = sub_div_element.text.strip()
        if ":" in sub_div_text:
            column_name, column_value = sub_div_text.split(":", 1)
            column_data[column_name] = [column_value]

    # Create a DataFrame from the collected data
    df = pd.DataFrame(column_data)

    # Add the "Stage" column with the component_title_text
    df["Stage"] = stage

    # Now, let's find the div element with class "rubber-body" within the main element
    rubber_body_element = main_element.find_element(By.CLASS_NAME, "rubber-body")

    # Find all tables with class "dc" within the rubber-body
    table_elements = rubber_body_element.find_elements(By.CLASS_NAME, "dc")

    # Initialize lists to store the information
    tables_data = []

    for table_element in table_elements:
        # Initialize data for each table
        table_data = {
            "Player": [],
            "Set 1": [],
            "Set 2": [],
            "Set 3": [],
            "Tie-Break 1": [],
            "Tie-Break 2": [],
            "Tie-Break 3": []
        }

        # Find the table body
        tbody_element = table_element.find_element(By.TAG_NAME, "tbody")

        # Find all rows (tr elements) within the tbody
        rows = tbody_element.find_elements(By.TAG_NAME, "tr")

        for row in rows:
            # Find all td elements within the row
            td_elements = row.find_elements(By.TAG_NAME, "td")

            # Extract and store the information starting from td_elements[1]
            player = td_elements[1].text.strip()

            # Extract results from td class "results"
            results = td_elements[2]
            set_scores = results.find_elements(By.TAG_NAME, "span")

            set_results = []
            tie_breaks = []

            for set_score in set_scores:
                set_result = set_score.text.strip()
                tie_break = ""

                # Use regular expressions to extract the first number in set_result
                match = re.search(r'\d+', set_result)
                if match:
                    set_result = match.group()
                else:
                    set_result = ""

                if set_score.find_elements(By.TAG_NAME, "sup"):
                    tie_break = set_score.find_element(By.TAG_NAME, "sup").text.strip()
                    # Use regular expressions to extract the first number in tie_break
                    match = re.search(r'\d+', tie_break)
                    if match:
                        tie_break = match.group()
                    else:
                        tie_break = ""

                set_results.append(set_result)
                tie_breaks.append(tie_break)

            # Ensure there are at most 3 sets
            set_results = set_results[:3]
            tie_breaks = tie_breaks[:3]

            # Assign the extracted values to the dictionary
            table_data["Player"].append(player)
            table_data["Set 1"].append(set_results[0])
            table_data["Set 2"].append(set_results[1])
            table_data["Set 3"].append(set_results[2])
            
            # Keep only the first element in the list for tie-breaks
            for i, tie_break in enumerate(tie_breaks):
                if i == 0 and tie_break:
                    table_data["Tie-Break 1"].append(tie_break)
                else:
                    table_data[f"Tie-Break {i+1}"].append(None)

        # Append the table data to the list
        tables_data.append(table_data)

    # Create a DataFrame from the collected data
    tables_df = pd.DataFrame(tables_data)

    # Combine the information from both DataFrames
    combined_df = pd.concat([df] * len(tables_df), ignore_index=True)
    combined_df = pd.concat([combined_df, tables_df], axis=1)

    # Add match and match status columns
    combined_df["match status"] = match_status
    combined_df["match"] = "match 1" 

    # Display the combined DataFrame
    print("Combined DataFrame:")
    print(combined_df)

except Exception as e:
    print("Error:", str(e))
finally:
    # Close the Selenium WebDriver
    driver.quit()


Combined DataFrame:
                    Date                                  Venue  \
0   02 Feb - 04 Feb 2018   Halle Olympique, Albertville, France   
1   02 Feb - 04 Feb 2018   Halle Olympique, Albertville, France   

                               Surface               Ball  \
0   Hard - Rebound Ace Synpave, Indoor   Tecnifibre X-One   
1   Hard - Rebound Ace Synpave, Indoor   Tecnifibre X-One   

                    Stage              Player Set 1 Set 2 Set 3 Tie-Break 1  \
0  WORLD GROUP  1ST ROUND  [Adrian MANNARINO]   [6]   [3]   [3]         [4]   
1  WORLD GROUP  1ST ROUND  [Thiemo DE BAKKER]   [7]   [6]   [6]         [7]   

  Tie-Break 2 Tie-Break 3        match status    match  
0      [None]      [None]  PLAYED & COMPLETED  match 1  
1      [None]      [None]  PLAYED & COMPLETED  match 1  
